In [26]:

# to do all the loss calculations, since automatic gradients are needed
import numpy as np

# Use helper packages
from AbstractBaseCollabFilterSGD import AbstractBaseCollabFilterSGD
from train_valid_test_loader import load_train_valid_test_datasets

from surprise import SVD
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate

# Some packages you might need (uncomment as necessary)
import pandas as pd
import matplotlib.pyplot as plt
print("hi")

hi


In [72]:
## Load the entire dev set in surprise's format
reader = Reader(
    line_format='user item rating', sep=',',
    rating_scale=(1, 5), skip_lines=1)

train_set = Dataset.load_from_file(
    '../data_movie_lens_100k/ratings_all_development_set.csv', reader=reader)

train_set = train_set.build_full_trainset()

# Use the SVD algorithm
n_factors = 50
## Fit model like our M3
model = SVD(n_factors=n_factors)
model.fit(train_set)

print("global mean:")
print(model.trainset.global_mean)
print("shape of bias_per_item: ")
print(model.bi.shape)
print("shape of bias_per_user: ")
print(model.bu.shape)
print("shape of U (per user vectors): ")
print(model.pu.shape) # pu is the user vector (ui)
print("shape of V (per item vectors): ")
print(model.qi.shape) # qi is the item vector (uv)


# 
user_info_df = pd.read_csv('../data_movie_lens_100k/user_info.csv')
movie_info_df = pd.read_csv('../data_movie_lens_100k/movie_info.csv')
movie_info_df = movie_info_df.drop('title', axis=1)

print("user info: ", user_info_df.shape)
print("movie shape: ", movie_info_df.shape)

# compare shapes

rated_movies = train_set.all_items()
rated_movie_ids = [int(train_set.to_raw_iid(i)) for i in rated_movies]  # Convert to raw ids

# Keep only movies in movie_info_df that have ratings
clean_movie_info_df = movie_info_df[movie_info_df['item_id'].isin(rated_movie_ids)]
print(f"Movies with ratings retained: {clean_movie_info_df.shape}")


# rename it so it's usable
users_U = model.pu
movies_V = model.qi
user_features = user_info_df
movie_features = clean_movie_info_df

ratings_y = train_set.all_ratings


global mean:
3.529480398257623
shape of bias_per_item: 
(1662,)
shape of bias_per_user: 
(943,)
shape of U (per user vectors): 
(943, 50)
shape of V (per item vectors): 
(1662, 50)
user info:  (943, 4)
movie shape:  (1681, 3)
Movies with ratings retained: (1662, 3)


In [73]:
# pipeline:

# 
# collapse U, V, user_feature, item_feature into one matrix to train

# put U and user_features together:
# Prepare user features
user_ids = list(range(0, 942))  # list of all user inner ids
user_features_list = []
for inner_id in user_ids:
    user_vector = users_U[inner_id]
    user_id = model.trainset.to_raw_uid(inner_id)
    user_meta = user_info_df.loc[user_info_df['user_id'] == int(user_id)].drop('user_id', axis=1)
    if not user_meta.empty:
        combined_features = np.concatenate([user_vector, user_meta.iloc[0].values])
        user_features_list.append(combined_features)

# Convert to DataFrame
user_features_df = pd.DataFrame(user_features_list, columns=[f'feature_{i}' for i in range(len(user_features_list[0]))])

# put V and item_features together:
# Prepare item features
item_ids = list(range(model.trainset.n_items))  # list of all item inner ids
item_features_list = []
for inner_id in item_ids:
    item_vector = model.qi[inner_id]
    item_id = model.trainset.to_raw_iid(inner_id)
    item_meta = movie_features.loc[movie_features['item_id'] == int(item_id)].drop('item_id', axis=1)
    if not item_meta.empty:
        combined_features = np.concatenate([item_vector, item_meta.iloc[0].values])
        item_features_list.append(combined_features)

# Convert to DataFrame
item_features_df = pd.DataFrame(item_features_list, columns=[f'feature_{i}' for i in range(len(item_features_list[0]))])



# after that for predcitions:

# on prediction:
    # take in user_id, item_id
    # get U and V from SVG algorithm 
    # get corresponding features of users/items
# condense the features and put into classifier for prediction

# get output of at least 4.5



In [74]:
# Generate user-item pairs with known ratings from your ratings dataset
ratings_df = pd.read_csv('../data_movie_lens_100k/ratings_all_development_set.csv')

# Merge user and item features into the ratings DataFrame
ratings_df = ratings_df.merge(user_features_df, left_on='user_id', right_index=True, how='left')
ratings_df = ratings_df.merge(item_features_df, left_on='item_id', right_index=True, how='left')

# print(ratings_df[0:5])

# Finding rows where any column has NaN

# Substitute NaN values with 0 in the entire DataFrame
ratings_df.fillna(0, inplace=True)

ratings_df.fillna(0, inplace=True)
cleaned_ratings = ratings_df
# Optionally, check the first few rows to confirm the substitution
print(ratings_df.head(5))

   user_id  item_id  rating  feature_0_x  feature_1_x  feature_2_x  \
0      772       36       3     0.000519    -0.018147    -0.010226   
1      471      228       5    -0.117474    -0.065134     0.133078   
2      641      401       4     0.030019    -0.019611     0.179792   
3      312       98       4     0.268444    -0.141399    -0.150881   
4       58      504       5    -0.178349    -0.058808     0.142644   

   feature_3_x  feature_4_x  feature_5_x  feature_6_x  ...  feature_42_y  \
0     0.192360    -0.209623    -0.008063     0.001900  ...      0.118245   
1     0.057704     0.008165     0.064068    -0.133663  ...     -0.047907   
2    -0.095667    -0.018864     0.261649     0.002808  ...     -0.065106   
3     0.110825    -0.042200     0.453833    -0.172197  ...      0.342833   
4    -0.073788     0.099370     0.134943    -0.149083  ...      0.053106   

   feature_43_y  feature_44_y  feature_45_y  feature_46_y  feature_47_y  \
0      0.168422      0.117078     -0.053176    

In [75]:

from sklearn.model_selection import train_test_split

feature_cols = [col for col in ratings_df.columns if col.startswith('feature')]

# Prepare data
X = cleaned_ratings[feature_cols]  # all your feature columns
y = cleaned_ratings['rating']      # or a binary column if you're classifying


# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
# Create a scaler object
scaler = StandardScaler()

# Fit on training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Initialize the KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5)

# Train the model
knn.fit(X_train_scaled, y_train)

# Predict on the test data
y_pred = knn.predict(X_test_scaled)

# Calculate the RMSE
rmse = np.mean(np.square(y_pred - y_test))
print(f'Root Mean Squared Error: {rmse}')




Root Mean Squared Error: 1.2201700094449692


In [88]:
# make prediction 

# pass in item_id, user_id into cool matrix thing
# Generate user-item pairs with known ratings from your ratings dataset
test_ratings_df = pd.read_csv('../data_movie_lens_100k/ratings_masked_leaderboard_set.csv')

# Merge user and item features into the ratings DataFrame
test_ratings_df = test_ratings_df.merge(user_features_df, left_on='user_id', right_index=True, how='left')
test_ratings_df = test_ratings_df.merge(item_features_df, left_on='item_id', right_index=True, how='left')

# Finding rows where any column has NaN
# cleaned_ratings = ratings_df.dropna()




X_test_true = test_ratings_df[feature_cols]
X_test_true = X_test_true.fillna(0)
print(X_test_true.shape)
y_test_true = test_ratings_df['rating']
X_test_true_scaled = scaler.transform(X_test_true)

# Predict on the test data
y_pred = knn.predict(X_test_true_scaled)

predicted_df = pd.read_csv('../data_movie_lens_100k/ratings_masked_leaderboard_set.csv')

predicted_df['rating'] = y_pred

binary_df = {
    'binary' : []
}

predicted_df['rating'] = predicted_df['rating'].astype(float)
# Create a new 'binary' column based on the condition
predicted_df['binary'] = np.where(predicted_df['rating'] >= 4.5, 1, 0)
# Export the DataFrame to a new CSV file
predicted_df['binary'].to_csv('predictions.csv', index=False)



(10000, 105)
